In [ ]:
# default_exp models.model

# Models

> Deep Learning modules with Fastai/Pytorch.

In [ ]:
# hide
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
# hide
from nbdev.showdoc import *

In [ ]:
# export
from steel_segmentation.core import *
from steel_segmentation.data import *
from steel_segmentation.preprocessing import *
from steel_segmentation.models.dls import *
from steel_segmentation.models.metrics import *
from steel_segmentation.models.unet import Unet

import warnings
with warnings.catch_warnings():
    warnings.simplefilter('ignore')
    from fastai.vision.all import *
    import fastai
from fastcore.foundation import *

In [ ]:
# hide
only_imgs = ["0a1cade03.jpg", "bca4ae758.jpg", "988cf521f.jpg", "b6a257b28.jpg",
             "b2ad335bf.jpg", "72aaba8ad.jpg", "f383950e8.jpg"]
train = train[train["ImageId"].isin(only_imgs)].copy()
train_all = train_all[train_all["ImageId"].isin(only_imgs)].copy()
train_multi = train_multi[train_multi["ImageId"].isin(only_imgs)].copy()

First we create a classification model to get an encoder that know how to classify defects pixels.
Then, we build a UNet from the trained encoder and train a segmentation model.

In [ ]:
# exports
models_dir = path.parent / "models"

In the Paperspace Gradient machine I stored these models:

In [ ]:
# missing
print_competition_data(models_dir)

../models/ResNet50_class.pt
../models/ResNet18_class.pt
../models/ResNet18-Unet-128-stage2.pth
../models/ResNet18-Unet-128-stage3.pth
../models/ResNet18-Unet-128-stage1.pth
../models/Resnet50-stage1.pth
../models/ResNet18-Unet-256-stage2.pth
../models/ResNet18-Unet-256-stage3.pth
../models/ResNet18-Unet-256-stage1.pth
../models/ResNet34-Unet-128-stage3.pth
../models/ResNet34-Unet-128-stage2.pth
../models/ResNet18_kaggle_class.pth
../models/ResNet34-stage1.pth
../models/ResNet18-stage1.pth
../models/ResNet34_class.pt
../models/kaggle_model.pth


## Classification

In [ ]:
# exports
class_metrics = [accuracy_multi, PrecisionMulti(), RecallMulti()]

These lines of code allow me to create a classification `Learner`.

In [ ]:
# missing
bs = 4

dls = get_classification_dls(bs)
arch = partial(resnet18, pretrained=True)
class_learner = cnn_learner(
    dls=dls, arch=arch, metrics=class_metrics, pretrained=True)

## Segmentation

In [ ]:
# exports
seg_metrics = [ModDiceMulti(), dice_kaggle]

These lines of code allow me to create a segmentation `Learner`.

In [ ]:
# missing
bs = 4
szs = (128, 800)

#dls = get_segmentation_dls_from_df(train_multi, bs, szs)
dls = get_segmentation_dls(bs, szs)
segmentation_learner = unet_learner(
    dls=dls, arch=resnet18, metrics=seg_metrics, pretrained=True)

To load a custom head in a Unet model:

In [ ]:
# missing
encoder_path = models_dir / "ResNet18-2_class.pt"
segmentation_learner.model[0].load_state_dict(
    torch.load(encoder_path), strict=True)

## Kaggle kernel

The code in this project is not only FastAi. I based an alternative solution based on this [kernel](https://www.kaggle.com/rishabhiitbhu/unet-starter-kernel-pytorch-lb-0-88). 
In this notebook I will go through each part of the model from that kernel.


In [ ]:
# export
import os
import pdb
import time
import warnings
import random
import numpy as np
import pandas as pd
from tqdm import tqdm_notebook as tqdm
from torch.optim.lr_scheduler import ReduceLROnPlateau
import torch
import torch.nn as nn
from torch.nn import functional as F
import torch.optim as optim
import torch.backends.cudnn as cudnn

warnings.filterwarnings("ignore")
seed = 69
random.seed(seed)
os.environ["PYTHONHASHSEED"] = str(seed)
np.random.seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True

In [ ]:
# cuda
model = Unet("resnet18", encoder_weights="imagenet", classes=4, activation=None)

In [ ]:
# cuda
model

Unet(
  (encoder): ResNetEncoder(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track

In [ ]:
# export
class Trainer:
    '''This class takes care of training and validation of our model'''
    
    def __init__(self, model, save_path,
                 num_epochs=20, lr=5e-4, 
                 bs=16, num_workers=6):
        self.num_workers = num_workers
        self.batch_size = {"train": bs, "val": bs//2}
        self.accumulation_steps = 32 // self.batch_size['train']
        self.lr = lr
        self.num_epochs = num_epochs
        self.net = model
        self.save_path = save_path
        
        self.best_loss = float("inf")
        self.phases = ["train", "val"]
        self.device = torch.device("cuda:0")
        torch.set_default_tensor_type("torch.cuda.FloatTensor")
        self.loss_fn = torch.nn.BCEWithLogitsLoss()
        self.optimizer = optim.Adam(self.net.parameters(), lr=self.lr)
        self.scheduler = ReduceLROnPlateau(self.optimizer, mode="min", patience=3, verbose=True)
        self.net = self.net.to(self.device)
        cudnn.benchmark = True
        
        self.dataloaders = {
            phase: get_kaggle_train_dls(
                data_folder=path,
                phase=phase,
                mean=(0.485, 0.456, 0.406),
                std=(0.229, 0.224, 0.225),
                batch_size=self.batch_size[phase],
                num_workers=self.num_workers,
            )
            for phase in self.phases
        }
        
        self.losses = {phase: [] for phase in self.phases}
        self.iou_scores = {phase: [] for phase in self.phases}
        self.dice_scores = {phase: [] for phase in self.phases}
        
    def forward(self, images, targets):
        """
        Forward pass: 
            load to GPU the imgs and masks,
            calculate predictions,
            calculate loss
        
        Returns:
            loss and predictions
        """
        images = images.to(self.device)
        masks = targets.to(self.device)
        preds = self.net(images)
        loss = self.loss_fn(preds, masks)
        return loss, preds

    def iterate(self, epoch, phase):
        """
        Iterate throught each batch in training or validatio phase.
        """
        meter = Meter(phase, epoch)
        start = time.strftime("%H:%M:%S")
        print(f"Starting epoch: {epoch} | phase: {phase} | ⏰: {start}")
        
        batch_size = self.batch_size[phase]
        self.net.train(phase == "train")
        dataloader = self.dataloaders[phase]
        
        running_loss = 0.0
        total_batches = len(dataloader)
#         tk0 = tqdm(dataloader, total=total_batches)
        self.optimizer.zero_grad()
        for itr, batch in enumerate(dataloader): # replace `dataloader` with `tk0` for tqdm
            images, targets = batch
            loss, outputs = self.forward(images, targets)
            loss = loss / self.accumulation_steps
            if phase == "train":
                loss.backward()
                if (itr + 1 ) % self.accumulation_steps == 0:
                    self.optimizer.step()
                    self.optimizer.zero_grad()
            running_loss += loss.item()
            outputs = outputs.detach().cpu()
            meter.update(targets, outputs)
#             tk0.set_postfix(loss=(running_loss / ((itr + 1))))

        epoch_loss = (running_loss * self.accumulation_steps) / total_batches
        dice, iou = epoch_log(phase, epoch, epoch_loss, meter, start)
        
        self.losses[phase].append(epoch_loss)
        self.dice_scores[phase].append(dice)
        self.iou_scores[phase].append(iou)
        
        torch.cuda.empty_cache()
        return epoch_loss

    def start(self):
        """
        Training loop for each epochs.
        """
        for epoch in range(self.num_epochs):
            self.iterate(epoch, "train")
            state = {
                "epoch": epoch,
                "best_loss": self.best_loss,
                "state_dict": self.net.state_dict(),
                "optimizer": self.optimizer.state_dict(),
            }
            with torch.no_grad():
                val_loss = self.iterate(epoch, "val")
                self.scheduler.step(val_loss)
            if val_loss < self.best_loss:
                print("******** New optimal found, saving state ********")
                state["best_loss"] = self.best_loss = val_loss
                torch.save(state, self.save_path)
            print()

In [ ]:
#show_doc(Trainer.__init__)

<h4 id="Trainer.__init__" class="doc_header"><code>Trainer.__init__</code><a href="__main__.py#L5" class="source_link" style="float:right">[source]</a></h4>

> <code>Trainer.__init__</code>(**`model`**)

Initialize self.  See help(type(self)) for accurate signature.

In [ ]:
#show_doc(Trainer.forward)

<h4 id="Trainer.forward" class="doc_header"><code>Trainer.forward</code><a href="__main__.py#L39" class="source_link" style="float:right">[source]</a></h4>

> <code>Trainer.forward</code>(**`images`**, **`targets`**)

Forward pass: 
    load to GPU the imgs and masks,
    calculate predictions,
    calculate loss

Returns:
    loss and predictions

In [ ]:
#show_doc(Trainer.iterate)

<h4 id="Trainer.iterate" class="doc_header"><code>Trainer.iterate</code><a href="__main__.py#L55" class="source_link" style="float:right">[source]</a></h4>

> <code>Trainer.iterate</code>(**`epoch`**, **`phase`**)

Iterate throught each batch in training or validatio phase.

In [ ]:
#show_doc(Trainer.start)

<h4 id="Trainer.start" class="doc_header"><code>Trainer.start</code><a href="__main__.py#L95" class="source_link" style="float:right">[source]</a></h4>

> <code>Trainer.start</code>()

Training loop for each epochs.

In [ ]:
# cuda
model_trainer = Trainer(model)

In [ ]:
# hide
from nbdev.export import notebook2script
notebook2script()

Converted 00_core.ipynb.
Converted 01_data.ipynb.
Converted 02_preprocessing.ipynb.
Converted 03_models.dls.ipynb.
Converted 04_model.metrics.ipynb.
Converted 05_models.unet.ipynb.
Converted 06_models.model.ipynb.
Converted index.ipynb.
